In [0]:
import base64

import databricks.sdk
from databricks.sdk.service.workspace import ImportFormat
import nbformat
import numpy as np 

import pandas as pd
import dspy

In [0]:
class Module(dspy.Module):
  def __init__(self, workspace_client, extract_code=False):
    self.workspace_client = workspace_client
    self.module_dict = None
    self.extract_code = extract_code

  def get_module_dict(self):
    return self.module_dict
  
  def set_module_dict(self, module_name):
    if module_name == "SQL":
      self.module_dict = SQLModule().get_module_dict()

  def get_code_from_notebooks(self, notebook_path):
    notebook = self.workspace_client.workspace.export(notebook_path, 
                                      format=ImportFormat.JUPYTER)
    ipynb = base64.decodebytes(notebook.content.encode('ascii')).decode("utf-8")
    notebook = nbformat.reads(ipynb, as_version=4)
    code_dict = {}
    x=0
    if self.extract_code:
      for cell in notebook.cells:
        if cell.cell_type == 'code':
          code_dict[x] = cell.source
          x+=1
    else:
      for cell in notebook.cells:
        if cell.cell_type == 'code' or cell.cell_type == 'markdown':
        # if cell.cell_type == 'code':
          code_dict[x] = cell.source
          x+=1

    context = ''.join(code_dict.values())
    
    return context
  
  # def get_code_from_notebooks_abhay(self, notebook_path):
  #   print("Using Abhay's method to extract code from notebooks")
  #   notebook = self.workspace_client.workspace.export(notebook_path, 
  #                                     format=ImportFormat.JUPYTER)
  #   ipynb = base64.decodebytes(notebook.content.encode('ascii')).decode("utf-8")
  #   lines = ipynb.split("\n")
  #   notebook_content = {}
  #   cell_index = 1
  #   current_cell = []
    
  #   for line in lines:
  #       if line.strip() in {"// COMMAND ----------", "# COMMAND ----------"}:
  #           if current_cell:  # Store previous cell
  #               notebook_content[f"cell{cell_index}"] = "\n".join(current_cell).strip()
  #               cell_index += 1
  #               current_cell = []
  #       else:
  #           current_cell.append(line)
    
    # # Add the last cell if not empty
    # if current_cell:
    #     notebook_content[f"cell{cell_index}"] = "\n".join(current_cell).strip()

    # return notebook_content
  
  def get_training_examples(self, table_name, instruction):
    df = spark.read.table(table_name)
    df = df.filter(trim(df.question) == instruction)
    training_examples = [dspy.Example(text=row.context, score=row.score, code_snippet = row.code_snippet).with_inputs("text") for row in df.collect()]
    return training_examples
  
  def evaluate_responses(self, context, few_shots_table_name):
    answer_list = []
    for k,module in self.module_dict.items():
      print(k, type(module))
      print(module)
      instruction = module.generate_answer.__dict__['predict'].__dict__['signature'].instructions
      print(instruction)

      training_examples = self.get_training_examples(few_shots_table_name, instruction)
      print(f" total training examples {len(training_examples)}")
      
    #   response = module(text=context)
    #   if self.extract_code:
    #     response_dict = {
    #       'reasoning': response.reasoning,
    #       'code_snippet': response.code_snippet,
    #       # 'question': response.code_snippet.desc
    #       'question': module.__dict__['predict'].__dict__['signature'].__fields__['code_snippet'].json_schema_extra['desc']
    #     }
    #   else:
    #     response_dict = {
    #       'question': module.__dict__['predict'].__dict__['signature'].instructions,
    #       'code_snippet': response.code_snippet,
    #       'score': response.score,
    #       # 'explanation': response.explanation,
    #       'chain_of_thought_reasoning': response.reasoning
    #     }
    #   # print(response_dict)
    #   answer_list.append(response_dict)
    # answer_list_df = pd.DataFrame(answer_list)

    # return answer_list_df
  
 
  
  def get_error_and_answer_dict(self, context_path, few_shots_table_name):
  
    context = self.get_code_from_notebooks(context_path)
    # Read from few shot db and add corresponding examples to few shot db
    
    answers = self.evaluate_responses(context, few_shots_table_name)
    # answers['context_url'] = context_path
    # if not self.extract_code:
    #   answers['score'] = answers['score'].astype('float64')
    # return {
    #   'context': context,
    #   'answers_dict': answers
    # }